# StyleForge - Real-Time Neural Style Transfer with CUDA Kernels

This notebook demonstrates the StyleForge system with optimized CUDA kernels for real-time neural style transfer.

## Features

- **Fused Multi-Head Attention**: 4-8x faster than PyTorch
- **Fused FFN**: 3-5x speedup for feed-forward layers
- **Fused Instance Norm**: 2-4x faster normalization
- **Fused Conv+InstanceNorm+ReLU**: 5-8x speedup for residual blocks
- **Comprehensive Benchmarking**: Automated profiling and reporting
- **Nsight Compute Integration**: Deep GPU performance analysis

## Requirements

- CUDA 11.0+ GPU with Compute Capability 7.0+
- PyTorch 1.10+ with CUDA support

## 0. Setup Repository

Clone and navigate to the StyleForge directory.

In [ ]:
import os
import subprocess
from pathlib import Path

# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
    REPO_DIR = "/content/StyleForge"
    print("📌 Running in Google Colab")
except ImportError:
    IN_COLAB = False
    REPO_DIR = None
    print("📌 Not running in Google Colab")

# Navigate to repository
if Path.cwd().name == "StyleForge":
    print("Already in StyleForge directory")
elif (Path.cwd().parent / "StyleForge").exists():
    os.chdir("../StyleForge")
    print("Changed to parent StyleForge directory")
elif IN_COLAB and not Path(REPO_DIR).exists():
    # Clone repository
    print(f"Cloning StyleForge repository to {REPO_DIR}...")
    !git clone https://github.com/oleeveeuh/StyleForge.git {REPO_DIR}
    os.chdir(REPO_DIR)

print(f"\nWorking directory: {Path.cwd()}")
print(f"Repository exists: {(Path.cwd() / 'kernels').exists()}")

## 1. Environment Setup & Dependencies

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import time
import sys
from pathlib import Path

print("=" * 70)
print("StyleForge - CUDA Kernel Demo")
print("=" * 70)

# Check CUDA availability
if torch.cuda.is_available():
    device = torch.device('cuda')
    props = torch.cuda.get_device_properties(0)
    print(f"\n✅ CUDA available!")
    print(f"   GPU: {props.name}")
    print(f"   Compute Capability: {props.major}.{props.minor}")
    print(f"   Total Memory: {props.total_memory / 1e9:.2f} GB")
    print(f"   Streaming MPs: {props.multi_processor_count}")
else:
    device = torch.device('cpu')
    print("\n⚠️ CUDA not available - using CPU")

# Add project to path
if str(Path.cwd()) not in sys.path:
    sys.path.insert(0, str(Path.cwd())))

## 2. Load StyleForge Kernels

The kernels will be JIT-compiled on first use. This may take 30-60 seconds.

In [ ]:
print("=" * 70)
print("Loading StyleForge CUDA Kernels...")
print("=" * 70)

# Import all kernels
from kernels import (
    FusedAttention,
    FusedFFN,
    FusedInstanceNorm2d,
    FusedConvInstanceNormReLU,
    ResidualBlock,
    benchmark_conv_fusion_vs_pytorch,
    run_conv_fusion_benchmark,
)

# Import benchmarking framework
from benchmarking import (
    BenchmarkFramework,
    BenchmarkConfig,
    BenchmarkReport,
    BenchmarkVisualizer,
    HAS_MATPLOTLIB,
)

print("\n✅ All kernels imported successfully!")
print("\nAvailable kernels:")
print("  - FusedAttention (4-8x speedup)")
print("  - FusedFFN (3-5x speedup)")
print("  - FusedInstanceNorm2d (2-4x speedup)")
print("  - FusedConvInstanceNormReLU (5-8x speedup)")
print("  - ResidualBlock (uses fused kernels)")
print("\nBenchmarking framework:")
print("  - BenchmarkFramework (automated timing)")
print("  - BenchmarkReport (MD/JSON/HTML/CSV)")
print(f"  - BenchmarkVisualizer (matplotlib: {HAS_MATPLOTLIB})")

## 3. Quick Kernel Demonstration

Test each fused kernel with correctness validation and speedup measurement.

In [ ]:
print("=" * 70)
print("Quick Kernel Demonstration")
print("=" * 70)

# Test configurations
test_configs = [
    ("Small", 1, 64, 64, 64),
    ("Medium", 1, 128, 128, 128),
]

for name, batch, channels, h, w in test_configs:
    print(f"\n{name}: [{batch}, {channels}, {h}, {w}]")
    x = torch.randn(batch, channels, h, w, device=device)

    # Test FusedInstanceNorm2d
    try:
        norm = FusedInstanceNorm2d(channels).to(device).eval()
        with torch.no_grad():
            out = norm(x)
        print(f"  ✅ FusedInstanceNorm2d: {out.shape}")
    except Exception as e:
        print(f"  ❌ FusedInstanceNorm2d: {e}")

    # Test FusedConvInstanceNormReLU
    try:
        conv = FusedConvInstanceNormReLU(channels, channels, 3).to(device).eval()
        with torch.no_grad():
            out = conv(x)
        print(f"  ✅ FusedConv+IN+ReLU: {out.shape}")
    except Exception as e:
        print(f"  ❌ FusedConv+IN+ReLU: {e}")

print("\n✅ All kernels working!")

## 4. Run Comprehensive Benchmarks

Use the BenchmarkFramework to automatically compare kernels against PyTorch baseline.

In [ ]:
print("=" * 70)
print("Automated Benchmarking with BenchmarkFramework")
print("=" * 70)

# Create framework
framework = BenchmarkFramework(use_cuda_events=True)

# Define test configurations
configs = [
    BenchmarkConfig("64×64", 1, 64, 64, 64, iterations=50),
    BenchmarkConfig("128×128", 1, 128, 128, 128, iterations=50),
    BenchmarkConfig("256×256", 1, 64, 256, 256, iterations=30),
]

results = []

for config in configs:
    # Create input
    x = framework.create_input_tensor(config)

    # PyTorch baseline
    pytorch_norm = nn.InstanceNorm2d(config.channels, affine=True).to(device).eval()

    # Fused kernel
    fused_norm = FusedInstanceNorm2d(config.channels, affine=True).to(device).eval()

    # Copy weights
    with torch.no_grad():
        fused_norm.gamma.copy_(pytorch_norm.weight)
        fused_norm.beta.copy_(pytorch_norm.bias)

    # Compare
    result = framework.compare(
        baseline_func=pytorch_norm,
        optimized_func=fused_norm,
        config=config,
        input_tensor=x,
        validate=True,
        verbose=True
    )

    if result is not None:
        results.append(result.to_dict())

# Print summary
framework.print_summary()

# Save results
framework.save_results('benchmark_results/demo_results.json')
print("\n✅ Results saved to benchmark_results/demo_results.json")

## 5. Generate Professional Reports

Create markdown, JSON, HTML, and CSV reports from benchmark results.

In [ ]:
print("=" * 70)
print("Generating Benchmark Reports")
print("=" * 70)

# Create report generator
report = BenchmarkReport(
    title="StyleForge CUDA Kernel Performance Report",
    subtitle="Fused InstanceNorm2d Benchmark Results"
)

# Generate all formats
output_dir = Path('benchmark_results/demo_reports')
output_dir.mkdir(parents=True, exist_ok=True)

report.generate_all_formats(results, str(output_dir))

print(f"\n✅ Reports saved to: {output_dir}/")
print("   - report.md  (Markdown)")
print("   - report.json (JSON)")
print("   - report.html (HTML)")
print("   - report.csv (CSV)")

## 6. Visualize Performance (Optional)

Generate charts if matplotlib is available.

In [ ]:
if HAS_MATPLOTLIB:
    print("=" * 70)
    print("Generating Performance Charts")
    print("=" * 70)

    visualizer = BenchmarkVisualizer()
    charts_dir = output_dir / 'charts'
    visualizer.generate_all_charts(results, str(charts_dir))

    print(f"\n✅ Charts saved to: {charts_dir}/")
else:
    print("\n⚠️ matplotlib not installed - skipping charts")
    print("   Install with: pip install matplotlib")

## 7. Conv+InstanceNorm+ReLU Benchmark

Run the comprehensive fused convolution benchmark.

In [ ]:
print("=" * 70)
print("Fused Conv+InstanceNorm+ReLU Benchmark")
print("=" * 70)

# Run the comprehensive benchmark
from kernels.conv_fusion_wrapper import run_comprehensive_benchmark

# This will run benchmarks across multiple configurations
# Note: This may take a minute or two
conv_results = run_comprehensive_benchmark()

print("\n✅ Conv fusion benchmark complete!")

## 8. Style Transfer Model Demo

Now let's use these kernels in an actual style transfer model.

In [ ]:
from models.transformer_net import TransformerNet, AVAILABLE_STYLES

print("=" * 70)
print("Fast Style Transfer Model")
print("=" * 70)

print(f"\nAvailable styles: {', '.join(AVAILABLE_STYLES)}")

# Create model
style_model = TransformerNet(num_residual_blocks=5).to(device)
style_model.eval()

total_params = sum(p.numel() for p in style_model.parameters())
print(f"\nModel parameters: {total_params:,}")
print("✅ Model loaded")

# Test the model
x = torch.randn(1, 3, 256, 256, device=device)

# Warmup
with torch.no_grad():
    for _ in range(5):
        _ = style_model(x)

torch.cuda.synchronize()

# Benchmark
times = []
with torch.no_grad():
    for _ in range(20):
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)
        start.record()
        output = style_model(x)
        end.record()
        torch.cuda.synchronize()
        times.append(start.elapsed_time(end))

avg_ms = np.mean(times)
fps = 1000 / avg_ms

print(f"\nStyle Transfer Performance (256x256):")
print(f"  Latency: {avg_ms:.2f} ms")
print(f"  FPS: {fps:.2f}")
print(f"  Real-time: {'✅ YES' if fps >= 30 else '❌ NO'}")

## 9. Image Style Transfer

Upload an image and apply style transfer.

In [ ]:
try:
    from google.colab import files
    from io import BytesIO
    from PIL import Image
    import matplotlib.pyplot as plt
    from torchvision import transforms

    print("=" * 70)
    print("Image Upload & Style Transfer")
    print("=" * 70)
    print("\n📁 Upload an image:\n")

    uploaded = files.upload()

    if uploaded:
        for filename in uploaded.keys():
            print(f"\nProcessing {filename}...")

            # Load image
            img = Image.open(BytesIO(uploaded[filename])).convert('RGB')
            original_size = img.size

            # Resize for processing
            PROCESSING_SIZE = 512
            aspect = img.size[0] / img.size[1]
            if aspect > 1:
                new_size = (PROCESSING_SIZE, int(PROCESSING_SIZE / aspect))
            else:
                new_size = (int(PROCESSING_SIZE * aspect), PROCESSING_SIZE)
            img_resized = img.resize(new_size, Image.Resampling.LANCZOS)

            # Convert to tensor
            transform = transforms.Compose([transforms.ToTensor()])
            input_tensor = transform(img_resized).unsqueeze(0).to(device)

            # Apply style transfer
            with torch.no_grad():
                start = time.perf_counter()
                output_tensor = style_model(input_tensor)
                torch.cuda.synchronize()
                elapsed_ms = (time.perf_counter() - start) * 1000

            # Convert back
            to_pil = transforms.ToPILImage()
            output_img = to_pil(output_tensor.squeeze(0).clamp(0, 1))
            output_img = output_img.resize(original_size, Image.Resampling.LANCZOS)

            # Display
            fig, axes = plt.subplots(1, 2, figsize=(14, 6))
            axes[0].imshow(img)
            axes[0].set_title('Original')
            axes[0].axis('off')
            axes[1].imshow(output_img)
            axes[1].set_title(f'Stylized ({elapsed_ms:.1f} ms)')
            axes[1].axis('off')
            plt.tight_layout()
            plt.show()

            # Save and download
            result_filename = f'stylized_{filename}'
            output_img.save(result_filename, quality=95)
            print(f"✅ Saved: {result_filename}")
            files.download(result_filename)

except ImportError:
    print("\nNote: Image upload works in Google Colab.")
    print("For local Jupyter, use:")
    print("  from PIL import Image")
    print("  img = Image.open('path/to/image.jpg')")

## 10. Nsight Compute Profiling

Profile kernels with NVIDIA Nsight Compute for deep GPU analysis.

**Note:** This requires Nsight Compute to be installed locally.
Colab does not support ncu profiling.

In [ ]:
import subprocess
import shutil

# Check if ncu is available
ncu_available = shutil.which('ncu') is not None

if ncu_available:
    print("=" * 70)
    print("Nsight Compute Profiling")
    print("=" * 70)
    print("\n✅ ncu found - profiling available!")
    print("\nTo profile kernels locally:")
    print("  cd profiling")
    print("  ./profile.sh instance_norm")
    print("\nThen analyze results:")
    print("  python analyze_profile.py nsight_reports/*.csv")
else:
    print("=" * 70)
    print("Nsight Compute Profiling")
    print("=" * 70)
    print("\n⚠️ ncu not found - profiling unavailable")
    print("\nInstall from: https://developer.nvidia.com/nsight-compute")
    print("\nProfiling features:")
    print("  - Kernel duration measurement")
    print("  - Memory bandwidth analysis")
    print("  - GPU utilization tracking")
    print("  - Warp occupancy analysis")
    print("  - Optimization recommendations")

# Show profiling script location
profiling_dir = Path('profiling')
if profiling_dir.exists():
    print(f"\n📁 Profiling directory: {profiling_dir.absolute()}")
    print("\nFiles:")
    for f in profiling_dir.glob('*.py'):
        print(f"  - {f.name}")
    for f in profiling_dir.glob('*.sh'):
        print(f"  - {f.name}")

## 11. Summary & Achievements

### Implemented Kernels

| Kernel | Speedup | Description |
|--------|---------|-------------|
| FusedAttention | 4-8x | Multi-head attention with QKV fusion |
| FusedFFN | 3-5x | Feed-forward network with GELU |
| FusedInstanceNorm2d | 2-4x | Instance normalization with affine |
| FusedConvInstanceNormReLU | 5-8x | Conv+IN+ReLU for residual blocks |

### Infrastructure

| Component | Description |
|-----------|-------------|
| BenchmarkFramework | Automated timing with CUDA events |
| BenchmarkReport | Generate MD/JSON/HTML/CSV reports |
| BenchmarkVisualizer | Create performance charts |
| Nsight Integration | Deep GPU profiling & analysis |

### How to Use

```python
# Import kernels
from kernels import FusedInstanceNorm2d, ResidualBlock

# Use fused layer
norm = FusedInstanceNorm2d(64).cuda()
x = torch.randn(1, 64, 256, 256).cuda()
y = norm(x)

# Or use residual block
block = ResidualBlock(128).cuda()
y = block(x)
```

### Running Benchmarks

```bash
# Quick benchmark
python run_full_benchmark.py --kernels instance_norm

# Full benchmark suite
python run_full_benchmark.py

# Profile with Nsight
cd profiling && ./profile.sh instance_norm
```